In [1]:
import urllib.request
import os
os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
from urllib.parse import urljoin
#import urllib
from urllib.parse import urlparse
import time
import glob
import pandas as pd
from pathlib import Path
#import pickle
#import psutil
#from zipfile import ZipFile
#import gc
#import matplotlib.pyplot as plt 
import rasterio
import rasterio.mask
from osgeo import gdal
from shutil import copyfile
import config

In [2]:
WIPLEX_config = config.WIPLEX_settings()
WIPLEX_config.initialize_config()
param_dict = WIPLEX_config.param
paths_dict = WIPLEX_config.paths

In [3]:
pa_gdf = gpd.read_file(f'{paths_dict["gwa_placement_area"]}', crs=param_dict["epsg_general"])

In [3]:
building_files = glob.glob(os.path.join(paths_dict["osm_out_path"], '*.geojson'))

In [4]:
building_files

['E:\\Master_Thesis\\WIPLEX\\Database\\Intermediate Outputs\\osm\\germany_4.geojson',
 'E:\\Master_Thesis\\WIPLEX\\Database\\Intermediate Outputs\\osm\\germany_5.geojson']

In [5]:
osm_gdf = gpd.read_file(building_files[2])

KeyboardInterrupt: 

In [40]:
%%time
join_list = []
for i in dfs:
    dfsjoin = gpd.sjoin(i, osm_gdf, how="left") #Spatial join Points to polygons
    counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count().to_numpy()
    join_list.append(counts)

CPU times: total: 3min 50s
Wall time: 4min


In [6]:
import itertools

In [10]:
%%time
for num, z in enumerate(building_files[:1]):
    print(f"Started calculating distance impacts for building batch {num + 1}/{len(building_files[:1])}")
    osm_gdf = gpd.read_file(z)

    inter_list = []
    for buff in [500]:
        print(f"Evaluating buffer zone {buff}m...")
        pa_buff = buffer_geoms(pa_gdf, buff , param_dict["epsg_distance_calc"])

        dfs = np.array_split(pa_buff, 8)

        count_list = []
        for i in dfs:
            dfsjoin = gpd.sjoin(i, osm_gdf, how="left") #Spatial join Points to polygons
            counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count().to_numpy()
            count_list.append(counts)
        count_list = list(itertools.chain(*count_list))
        inter_list.append(count_list)


    np.save(f'{paths_dict["osm_out_path"]}/impacts_{num}', np.array(inter_list))
    #impact_list.append(np.array(inter_list))
    print(f"Finished {num + 1}. batch. Time passed:", time.process_time() - start)

Started calculating distance impacts for building batch 1/1
Evaluating buffer zone 500m...


NameError: name 'start' is not defined

In [17]:
test = np.load(f'{paths_dict["osm_out_path"]}/impacts_0.npy')

In [19]:
%%time
inter_list = [] #4min 25s
for buff in [500]:
    print(f'Calculating {buff}m impacts...')
    pa_buff = buffer_geoms(pa_gdf, buff , param_dict["epsg_distance_calc"])

    dfs = np.array_split(pa_buff, 10)
    
    count_list = []
    for i in dfs:
        dfsjoin = gpd.sjoin(i, osm_gdf, how="left", predicate="intersects") #Spatial join Points to polygons
        counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count().to_numpy()
        count_list.append(counts)
    count_list = list(itertools.chain(*count_list))
    inter_list.append(count_list) # add np.array of intersection count for file z to a list

Calculating 500m impacts...
CPU times: total: 4min 28s
Wall time: 4min 28s


In [ ]:
def add_house_impact_allocation(param_dict, paths_dict):
    '''
    Function counts the number of affected buildings for a GeoDataFrame with wind turbine locations given a set of 
    impact zones in meter. File locations and the set of impact zones should be specified in the config.py file before
    running this function.

    Parameters
    ----------
        param_dict : dict
            parameter dictionary specified in the config.py file
        paths_dict : dict
            paths dictionary specified in the config.py file

    Returns
    -------
        None
    '''
    print('Started calculating impact allocation')
    start = time.process_time() # start time check for calculations
    
    building_files = glob.glob(os.path.join(paths_dict["osm_out_path"], '*.geojson'))
    pa_gdf = gpd.read_file(f'{paths_dict["gwa_placement_area"]}', crs=param_dict["epsg_general"])
    

    impact_list = []
    for num, z in enumerate(building_files):
        print(f"Started calculating distance impacts for building batch {num + 1}/{len(building_files)}")
        osm_gdf = gpd.read_file(z)

        inter_list = []
        for buff in param_dict["impact_buffers"]:
            print(f"Evaluating buffer zone {buff}m...")
            pa_buff = buffer_geoms(pa_gdf, buff , param_dict["epsg_distance_calc"])

            dfs = np.array_split(pa_buff, 5)

            count_list = []
            for i in dfs:
                dfsjoin = gpd.sjoin(i, osm_gdf, how="left") #Spatial join Points to polygons
                counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count().to_numpy()
                count_list.append(counts)
            count_list = list(itertools.chain(*count_list))
            inter_list.append(count_list)

        
        impact_list.append(np.array(inter_list))
        print(f"Finished {num}. batch. Time passed:", time.process_time() - start)
    
    # Correct impact double-counting:
    # To avoid double counting of impacts subtract the larger buffer zone by the next smaller one
    # The smallest buffer zone impact count will simply be subtracted by 0 
    impacts_dc = np.sum(impact_list, axis=0).T
    sub_array= np.hstack((np.zeros((impacts_dc.shape[0],1)), impacts_dc[:, :-1])) # adds zero column to numpy array in position 0
    corrected_impacts = impacts_dc - sub_array
    impact_names = [f"imp_{buff}" for buff in param_dict["impact_buffers"]]
    pa_gdf[impact_names] = corrected_impacts.astype(int)


    # Save file:
    pa_gdf.to_file(f'{paths_dict["optimization_file"]}')
    print('Finished generating impact allocation')

    return None

In [10]:
filepath = "E:/Master_Thesis/WIPLEX/Database/Input Files/osm/arnsberg-regbez-latest-free.shp"

In [11]:


osm_gdf = gpd.read_file(f"{filepath}/{filename}").loc[:,['geometry']].to_crs(param_dict["epsg_distance_calc"])
print(f"Time", time.process_time() - start)
# 256.46875

Time 273.796875


In [5]:
filename = paths_dict["osm_file_names"]["buildings"]
osmfiles = glob.glob(os.path.join(paths_dict["osm_path"], '*.zip'))
file_size = 0
numbering = 1
df_out = gpd.GeoDataFrame()
start = time.process_time()
for z in osmfiles:
    file_size += os.path.getsize(z)/1000000
    osm_gdf = gpd.read_file(f"zip://{z}/{filename}").loc[:,['geometry']].to_crs(param_dict["epsg_distance_calc"])
    osm_gdf = osm_gdf.set_geometry(osm_gdf.centroid).to_crs(param_dict["epsg_general"])
    
    gdf_res = gpd.read_file(f"zip://{z}/{paths_dict['osm_file_names']['landuse']}").loc[:, ['fclass','geometry']]
    gdf_res = gdf_res.loc[(gdf_res['fclass'] == 'residential')]
    osm_gdf = gpd.clip(osm_gdf, gdf_res)
    df_out = pd.concat([df_out, osm_gdf])
    print("Finished Processing file:", z)
    print(f"Time", time.process_time() - start)
    if file_size > 1000:
        df_out.to_file(f'germany_{numbering}.geojson', driver='GeoJSON')
        numbering += 1
        file_size = 0
        df_out = gpd.GeoDataFrame()

Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\arnsberg-regbez-latest-free.shp.zip
Time 537.4375
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\berlin-latest-free.shp.zip
Time 693.96875
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\brandenburg-latest-free.shp.zip
Time 1359.703125
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\bremen-latest-free.shp.zip
Time 1411.46875
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\detmold-regbez-latest-free.shp.zip
Time 1734.890625
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\duesseldorf-regbez-latest-free.shp.zip
Time 2356.765625
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\freiburg-regbez-latest-free.shp.zip
Time 3228.140625
Finished Processing file: E:\Master_Thesis\WIPLEX\Database\Input Files\osm\hamburg-latest-free.shp.zip
Time 3327.75
Finished Proce

In [26]:
df_out = gpd.GeoDataFrame()
test = pd.concat([df_out, osm_gdf])

In [3]:
pa_gdf = gpd.read_file(f'{paths_dict["gwa_placement_area"]}', crs=param_dict["epsg_general"])

In [5]:
building_files = glob.glob(os.path.join(paths_dict["osm_out_path"], '*.geojson'))
test_buildings = gpd.read_file(building_files[0])

In [10]:
import haversine as hs

In [23]:
loc1 = test_buildings.iloc[0]["geometry"]
loc2 = pa_gdf.iloc[0]["geometry"]
#hs.haversine(loc1,loc2)
loc1.distance(loc2)

3.3790047163470738

In [56]:
test_gdf = pa_gdf.copy()

In [44]:
from scipy.spatial.distance import cdist

In [8]:
def hs_distance(point1, point2):
    dist = hs.haversine((point1.x, point1.y), (point2.x, point2.y), unit=hs.Unit.METERS)
    return int(dist)

def buffer_geoms(gdf_in, buffer, proj_epsg=25832):
    '''
    Function calculates a buffered GeoSeries from the geometry column of a geopandas DataFrame
    in a given meter based coordinate reference system (crs) and converts it back to the orginial crs
    '''
    init_crs = gdf_in.crs
    gdf_in = gdf_in.to_crs(proj_epsg)
    buffered = gpd.GeoDataFrame(geometry = gdf_in.geometry.buffer(buffer)).to_crs(init_crs)

    return buffered

def intersection_counter(gdf_main, gdf_intersect):
    '''
    Function counting for every row in the gdf_main the number of intersections with the rows in gdf_intersect
    Input: gdf_main := GeoDataFrame which is looped over counting the intersections for each row
           gdf_intersect := GeoDataFrame with the intersecting geometries 
           
    Output: intersect_count := List of tuples containing the row index number in the gdf_main as first element 
                               and the number of intersections for each row 
                               
    Useful Explanation: https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
    '''
    intersect_count = []
    spatial_index = gdf_intersect.sindex
    for index, row in gdf_main.iterrows():
        possible_matches_index = list(spatial_index.intersection(row['geometry'].bounds))
        possible_matches = gdf_intersect.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(row['geometry'])]
        intersect_count.append((index, len(precise_matches)))
        
    return intersect_count

In [134]:
pa_buff = buffer_geoms(pa_gdf, 3500, param_dict["epsg_distance_calc"])

In [136]:
%%time
dfsjoin = gpd.sjoin(pa_buff, test_buildings, how="left") #Spatial join Points to polygons

KeyboardInterrupt: 

In [112]:
counts2 = dfsjoin2.groupby(dfsjoin2.index)["index_right"].count().to_numpy()

In [130]:
counts = dfsjoin.groupby(dfsjoin.index)["index_right"].count()#.to_numpy()

In [108]:
# test = counts[counts != counts2]
# test2 = counts2[counts != counts2]
# for i, j in enumerate(test2):
#     print(j, test[i])


In [120]:
counts3 = dfsjoin2.groupby(dfsjoin2["index_right"]).count().to_numpy()

In [126]:
test = dfsjoin2.groupby(dfsjoin2["index_right"]).count()

In [132]:
counts.iloc[45832]

2

In [127]:
test

,geometry
index_right,
45832.0,2
46295.0,27
46296.0,101
46297.0,61
46298.0,1
...,...
311542.0,18
311543.0,9
312078.0,1


In [135]:
%%time
dfsjoin2 = gpd.sjoin(test_buildings, pa_buff, predicate="within", how="left") #Spatial join Points to polygons

KeyboardInterrupt: 

In [ ]:
%%time
counted2 = np.array(intersection_counter(pa_gdf, test_buildings))

In [24]:
dm = cdist()

7.873770818612805

In [78]:
pa_test = pa_gdf.copy()
pa_test = pa_test.to_crs(param_dict["epsg_distance_calc"])

In [80]:
test_buildings2 = test_buildings.copy()
test_buildings2 = test_buildings2.to_crs(param_dict["epsg_distance_calc"])

In [81]:
%%time
test = pa_test.geometry.apply(lambda g: test_buildings2.distance(g))

<timed exec>:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



KeyboardInterrupt: 

In [13]:
osm_gdf = osm_gdf.set_geometry(osm_gdf.centroid).to_crs(param_dict["epsg_general"])

In [6]:
start = time.process_time()
gpd.read_file("germany_1.geojson")
print(f"Time", time.process_time() - start)

Time 448.9375


In [16]:
osm_gdf.to_file('myJson.geojson', driver='GeoJSON')

In [31]:
# test = list(pa_gdf.columns)
# test.remove("index")
# test2 = list(pa_gdf_start.columns)
order = list(pa_gdf_start.columns) + [item for item in list(pa_gdf.columns) if not item in list(pa_gdf_start.columns)]
pa_gdf = pa_gdf[order]
pa_gdf.drop(columns=["index"], inplace=True)
pa_gdf.columns

Index(['ad_100', 'ad_150', 'ad_200', 'ad_50', 'iec1', 'iec2', 'iec3', 'ws_100',
       'ws_150', 'ws_200', 'ws_50', 'geometry', 'imp_500m', 'imp_750m',
       'imp_1000m', 'imp_1250m', 'imp_1500m', 'imp_1750m', 'imp_2000m',
       'imp_2250m', 'imp_2500m', 'imp_2750m', 'imp_3000m', 'imp_3250m',
       'imp_3500m', 'imp_3750m', 'imp_4000m', 'imp_4250m', 'imp_4500m',
       'imp_4750m', 'imp_5000m', 'imp_5250m', 'imp_5500m'],
      dtype='object')

In [29]:
impacts_dc = pa_gdf.iloc[:, col_len:].copy().to_numpy()
sub_array= np.hstack((np.zeros((impacts_dc.shape[0],1)), impacts_dc[:, :-1])) # adds zero column to numpy array in position 0
corrected_impacts = impacts_dc - sub_array
pa_gdf.iloc[:, col_len:] = corrected_impacts.astype(int)